In [18]:
"""Import"""
import os 
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
from torchvision import datasets, transforms , models
from torch.utils.data import DataLoader
import torch
import torch.optim  as optim


import numpy as np
from PIL import Image
from skimage import io
from sklearn.model_selection import train_test_split






In [3]:
WIDTH = 224
HEIGHT = 224
NUM_CHANNELS = 3
NUM_CLASSES = 10
NUM_EPOCHS = 10
LR = 1e-4
BATCH_SIZE = 32

In [4]:
test_images = []
images_path = './dataset/'
test_images_pd = pd.read_csv("test.csv")
test_images_pd["Image"] = test_images_pd["Image"].astype("str").str.zfill(5)

for i in test_images_pd.values:
    image_name = i[0]
    image_path = os.path.join(images_path, image_name+'.jpg')
    test_images.append(image_path)

In [5]:
transform = transforms.Compose([
    transforms.Resize((WIDTH, HEIGHT)),  
    transforms.ToTensor()         
])


transformed_test_images = []

for img_name in test_images:
    transformed_img = transform(Image.open(img_name))
    transformed_test_images.append(transformed_img)


transformed_test_images = torch.stack(transformed_test_images)

In [6]:
data = pd.read_csv("train.csv")
data["Image"] = data["Image"].astype("str").str.zfill(5)
images_path = "./dataset/"
images_list = []
labels_list = []
for i in data.values:
    image_name, label = i
    images_list.append(os.path.join(images_path,image_name+".jpg"))
    labels_list.append(label)



In [7]:
train_files, val_files, train_labels, val_labels = train_test_split(
        images_list, labels_list, test_size=0.1, stratify=labels_list)

In [8]:

transform = transforms.Compose([transforms.ToTensor(),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(20),
            transforms.RandomResizedCrop(128),
            transforms.Resize((WIDTH,HEIGHT)),
    
            ])

In [9]:
class Dataset:
    def __init__(self,images_list,labelss):
        self.labels = torch.from_numpy(np.array(labelss,dtype = np.float64)).type(torch.float64)
        self.images = [io.imread(image) for image in images_list]
        self.transform = transform
    def __getitem__(self,index):
        image = self.images[index]
        image = self.transform(image)
        label = self.labels[index]
        return image,label
    def __len__(self):
        return len(self.labels)

In [10]:
train_dataset = Dataset(images_list=train_files,labelss=train_labels)
valid_dataset = Dataset(images_list=val_files,labelss=val_labels)

train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE)
val_loader = DataLoader(valid_dataset,batch_size=BATCH_SIZE)
test_loader = DataLoader(transformed_test_images, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
class Cnn_Model(nn.Module):
    def __init__(self):
        super(Cnn_Model,self).__init__()
        # input (m,3,224,224)
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=128,kernel_size=3), # (m,128,222,222)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2,2), # (m,128,111,111)
            
            nn.Conv2d(in_channels=128,out_channels=64,kernel_size=3), # (m,64,109,109)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2,2), # (m, 64, 54, 54)

            nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3), # (m, 32, 52, 52) 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2,2), #(m,32,26,26)

            nn.Conv2d(in_channels=32,out_channels=16,kernel_size=3),#(m,16,24,24)
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2,2),#(m,16,12,12)
            nn.Flatten(), # (m ,16*12*12)
            nn.Linear(in_features=16*12*12,out_features=128), #(m ,128)
            nn.ReLU(),
            nn.Linear(in_features=128,out_features=NUM_CLASSES),
            # nn.LogSoftmax(dim=1)
        )
    def forward(self, x):
        x = self.model(x)
        return x
        

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Cnn_Model().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained = True)
# for param in model.parameters():
#     param.requires_grad = False
# model.classifier[4] = nn.Linear(4096,1024)
# model.classifier[6] = nn.Linear(1024,10)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,NUM_CLASSES)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LR)

In [50]:
def early_stopping(train_loss, validation_loss, min_delta, tolerance):

    counter = 0
    if (validation_loss - train_loss) > min_delta:
        counter +=1
        if counter >= tolerance:
          return True


In [ ]:
n_total_steps = len(train_loader)
print(n_total_steps)
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        labels=labels.to(torch.int64)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [15]:
def predict(model, loader):
  model.eval()
  preds = []
  with torch.no_grad():
    for images in loader:
      images = images.to(device)
      # Forward pass
      outputs = model(images).detach().cpu().numpy()
      preds.extend(list(np.argmax(outputs, axis=1)))
  return np.array(preds)

In [16]:
a = predict(model,test_loader)
a

array([5, 3, 2, 8, 4, 6, 3, 8, 4, 9, 4, 3, 6, 3, 6, 1, 9, 0, 4, 9, 3, 0,
       1, 5, 0, 8, 3, 4, 5, 9, 1, 4, 4, 3, 1, 4, 0, 4, 7, 3, 3, 4, 2, 3,
       6, 5, 1, 4, 5, 4, 4, 4, 9, 4, 8, 7, 3, 2, 9, 9, 4, 3, 6, 2, 6, 9,
       2, 4, 7, 8, 6, 3, 3, 4, 3, 9, 4, 6, 3, 3, 4, 5, 6, 2, 3, 4, 3, 4,
       4, 9, 0, 7, 9, 9, 8, 1, 1, 8, 9, 4, 7, 1, 3, 7, 5, 4, 3, 1, 1, 4,
       7, 7, 4, 2, 7, 4, 4, 1, 2, 6, 3, 8, 2, 0, 8, 3, 9, 3, 4, 0, 6, 3,
       6, 7, 7, 3, 4, 7, 9, 8, 7, 3, 1, 0, 4, 9, 7, 7, 0, 2, 1, 4, 2, 9,
       8, 0, 4, 1, 7, 0, 3, 9, 0, 4, 2, 4, 3, 4, 2, 1, 0, 1, 6, 8, 8, 7,
       4, 1, 4, 9, 0, 2, 3, 2, 2, 2, 7, 9, 2, 1, 7, 6, 4, 2, 7, 3, 2, 7,
       3, 5, 4, 1, 5, 1, 0, 8, 4, 7, 7, 4, 1, 4, 8, 3, 9, 2, 5, 5, 4, 7,
       1, 1, 4, 0, 6, 2, 9, 4, 2, 6, 6, 1, 9, 1, 5, 6, 4, 7, 1, 9, 1, 9,
       6, 6, 3, 2, 3, 6, 9, 1, 4, 3, 7, 4, 2, 7, 4, 6, 7, 3, 7, 1, 9, 6,
       9, 4, 4, 9, 6, 3, 6, 8, 1, 4, 4, 1, 6, 3, 4, 6, 3, 4, 5, 3, 0, 4,
       4, 5, 5, 2, 6, 9, 9, 3, 7, 8, 4, 1, 6, 5, 7,

In [17]:
def create_submission(predictions, filename):
    with open(filename + '.csv', 'w') as solution_file:
        solution_file.write('Id,Predicted\n')
        for i, string in enumerate(predictions):
            solution_file.write(str(i)+","+str(string)+'\n')

create_submission(a, "sample_submission")